In [1]:
import argparse
import logging
import os
import sys

import numpy as np
from tqdm import tqdm
from matplotlib import pyplot as plt

import torch
import torch.nn as nn
from torch import optim
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader, random_split

sys.path.append("..")
sys.path.append("../scripts/")
import superlayer.utils

from scripts import eval_net, train_net, get_args

from superlayer.models import SLNet, SUnet, SL_AEnet, AEnet
from superlayer.utils import BrainD, dice_coeff, one_hot, plot_img_array, plot_side_by_side

In [2]:
dir_img = '/home/gid-dalcaav/projects/neuron/data/t1_mix/proc/resize256-crop_x32-slice100/train/vols/'
dir_mask = '/home/gid-dalcaav/projects/neuron/data/t1_mix/proc/resize256-crop_x32-slice100/train/asegs/'

dir_train = '/home/vib9/src/SL-Net/jupyter/partitions/train.txt'
dir_val = '/home/vib9/src/SL-Net/jupyter/partitions/val.txt'

dir_checkpoint_1 = 'checkpoints_1/'
dir_checkpoint_2 = 'checkpoints_2/'

In [3]:
logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')
args = get_args()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
logging.info(f'Using device {device}')

INFO: Using device cuda


In [4]:
target_label_numbers = [0,2,3,4,10,16,17,28,31,41,42,43,49,53,63]
val_percent = 0.1
batch_size = args.batchsize
lr = args.lr
img_scale = args.scale
writer = SummaryWriter(comment=f'LR_{lr}_BS_{batch_size}_SCALE_{img_scale}')
enc_nf = [64, 64, 64, 64]
dec_nf = [64, 64, 64, 64]

In [5]:
net1 = AEnet(input_ch=1, out_ch=15, use_bn=True, enc_nf=enc_nf, dec_nf=dec_nf)

logging.info(f'Network:\n'
             f'\t{1} input channels\n'
             f'\t{15} output channels (classes)\n')

net1.to(device=device)
train_scores1, val_scores1, train_var_1, val_var_1 = train_net(net=net1,
                                                                epochs=5,
                                                                batch_size=args.batchsize,
                                                                lr=args.lr,
                                                                device=device,
                                                                img_scale=args.scale,
                                                                val_percent=args.val / 100,
                                                                checkpoint=2,
                                                                target_label_numbers=target_label_numbers,
                                                                writer=writer,
                                                                train_path=dir_train,
                                                                val_path=dir_val)

INFO: Network:
	1 input channels
	15 output channels (classes)

INFO: Creating dataset with 7329 examples
INFO: Creating dataset with 7329 examples
INFO: Starting training:
        Epochs:          5
        Batch size:      8
        Learning rate:   0.001
        Training size:   7329
        Validation size: 7329
        Checkpoints:     True
        Device:          cuda
        Images scaling:  1
    
Validation round:  72%|███████▏  | 657/916 [00:27<00:12, 21.16batch/s]


Validation round: 100%|█████████▉| 913/916 [00:38<00:00, 20.89batch/s]
                                                                      INFO: Validation Dice Loss: 0.15105320847548512
Validation round:  45%|████▌     | 415/916 [00:18<00:20, 24.24batch/s]


Validation round:  99%|█████████▉| 911/916 [00:38<00:00, 23.16batch/s]
                                                                      INFO: Validation Dice Loss: 0.12180431564562186
Validation round:  23%|██▎       | 211/916 [00:09<00:29, 24.24batch/s]


Validation round: 100%|█████████▉| 915/916 [00:38<00:00, 24.63batch/s]
                                                                      INFO: Validation Dice Loss: 0.10973277750656854
Validation round:   5%|▍         | 43/916 [00:02<00:51, 16.86batch/s]


Validation round:  71%|███████▏  | 654/916 [00:26<00:13, 19.72batch/s]


...

Validation round: 100%|█████████▉| 912/916 [00:50<00:00, 16.34batch/s]
                                                                      INFO: Validation Dice Loss: 0.07220541602041264
Validation round:  30%|███       | 278/916 [00:15<00:31, 20.00batch/s]


Validation round:  82%|████████▏ | 753/916 [00:41<00:07, 20.46batch/s]


Validation round: 100%|█████████▉| 915/916 [00:49<00:00, 20.67batch/s]
                                                                      INFO: Validation Dice Loss: 0.07152138551184704
Validation round:  30%|██▉       | 274/916 [00:15<00:31, 20.56batch/s]


Validation round:  82%|████████▏ | 750/916 [00:41<00:10, 16.33batch/s]


Validation round: 100%|█████████▉| 914/916 [00:50<00:00, 20.47batch/s]
                                                                      INFO: Validation Dice Loss: 0.07458622007748705
Epoch 5/5: 100%|██████████| 7329/7329 [05:20<00:00, 22.89img/s, loss (batch)=0.0979]
INFO: Checkpoint 5 saved !


In [ ]:
net2 = SL_AEnet(input_ch=1, out_ch=15, use_bn=True, superblock_size=64, depth=4)

logging.info(f'Network:\n'
             f'\t{1} input channels\n'
             f'\t{15} output channels (classes)\n')

net2.to(device=device)
train_scores2, val_scores2, train_var_2, val_var_2 = train_net(net=net2,
                                                                epochs=5,
                                                                batch_size=args.batchsize,
                                                                lr=args.lr,
                                                                device=device,
                                                                img_scale=args.scale,
                                                                val_percent=args.val / 100,
                                                                checkpoint=2,
                                                                target_label_numbers=target_label_numbers,
                                                                writer=writer,
                                                                train_path=dir_train,
                                                                val_path=dir_val)

INFO: Network:
	1 input channels
	15 output channels (classes)

INFO: Creating dataset with 7329 examples
INFO: Creating dataset with 7329 examples
INFO: Starting training:
        Epochs:          5
        Batch size:      8
        Learning rate:   0.001
        Training size:   7329
        Validation size: 7329
        Checkpoints:     True
        Device:          cuda
        Images scaling:  1
    
Validation round:  46%|████▋     | 424/916 [00:23<00:26, 18.41batch/s]


Validation round:  94%|█████████▍| 859/916 [00:47<00:03, 14.56batch/s]


Validation round: 100%|█████████▉| 913/916 [00:49<00:00, 20.40batch/s]
                                                                      INFO: Validation Dice Loss: 0.15018474639242912
Validation round:  41%|████▏     | 379/916 [00:21<00:33, 15.89batch/s]


Validation round:  89%|████████▉ | 816/916 [00:43<00:04, 22.44batch/s]


Validation round: 100%|█████████▉| 914/916 [00:48<00:00, 19.78batch/s]
                                                                      INFO: Validation Dice Loss: 0.15446695609742117
Validation round:  39%|███▊      | 354/916 [00:21<00:44, 12.57batch/s]


...

Validation round:  53%|█████▎    | 488/916 [00:24<00:17, 23.91batch/s]


Validation round: 100%|█████████▉| 914/916 [00:43<00:00, 21.87batch/s]
                                                                      INFO: Validation Dice Loss: 0.09103507167907782
Epoch 3/5: 100%|██████████| 7329/7329 [04:32<00:00, 26.85img/s, loss (batch)=0.155] 
INFO: Checkpoint 3 saved !
Validation round:   4%|▍         | 39/916 [00:02<01:15, 11.69batch/s]


Validation round:  60%|██████    | 551/916 [00:27<00:19, 18.78batch/s]


Validation round:  99%|█████████▉| 911/916 [00:43<00:00, 23.19batch/s]
                                                                      INFO: Validation Dice Loss: 0.08836569381658167
Validation round:  15%|█▍        | 136/916 [00:06<00:28, 27.17batch/s]


Validation round:  43%|████▎     | 395/916 [00:19<00:27, 19.15batch/s]

In [ ]:
net3 = SUnet(input_ch=1, out_ch=15, use_bn=True, enc_nf=enc_nf, dec_nf=dec_nf, ignore_last=False)

logging.info(f'Network:\n'
             f'\t{1} input channels\n'
             f'\t{15} output channels (classes)\n')

net3.to(device=device)
train_scores3, val_scores3, train_var_3, val_var_3 = train_net(net=net3,
                                                                epochs=5,
                                                                batch_size=args.batchsize,
                                                                lr=args.lr,
                                                                device=device,
                                                                img_scale=args.scale,
                                                                val_percent=args.val / 100,
                                                                checkpoint=1,
                                                                target_label_numbers=target_label_numbers,
                                                                writer=writer,
                                                                train_path=dir_train,
                                                                val_path=dir_val)

In [ ]:
net4 = SLNet(input_ch=1, out_ch=15, use_bn=True, superblock_size=64, depth=4)

logging.info(f'Network:\n'
             f'\t{1} input channels\n'
             f'\t{15} output channels (classes)\n')

net4.to(device=device)
train_scores4, val_scores4, train_var_4, val_var_4 = train_net(net=net4,
                                                                epochs=5,
                                                                batch_size=args.batchsize,
                                                                lr=args.lr,
                                                                device=device,
                                                                img_scale=args.scale,
                                                                val_percent=args.val / 100,
                                                                checkpoint=2,
                                                                target_label_numbers=target_label_numbers,
                                                                writer=writer,
                                                                train_path=dir_train,
                                                                val_path=dir_val)

In [ ]:
print("configuring combined plots")
domain = len(train_scores4)

x_values = [i+1 for i in range(domain)]
a1 = plt.subplot(2,2,1)
a1.set_ylim([0, 0.5])
plt.title("Simple TiedNet: 2,952,448 params")
plt.xlabel("Mini-epochs")
plt.ylabel("Dice Loss")
plt.plot(x_values, train_scores4, color="blue", label="train")
plt.plot(x_values, val_scores4, color="orange", label="val")
plt.legend()
plt.grid()

domain = len(train_scores3)
a2 = plt.subplot(2,2,2)
a2.set_ylim([0, 0.5])
plt.title("Simple Net: 7,117,071 params")
plt.xlabel("Mini-epochs")
plt.ylabel("Dice Loss")
plt.plot(x_values, train_scores3, color="blue", label="train")
plt.plot(x_values, val_scores3, color="orange", label="val")
plt.legend()
plt.grid()

domain = len(train_scores1)
a2 = plt.subplot(2,2,3)
a2.set_ylim([0, 0.5])
plt.title("Simple AENet: 7,117,071 params")
plt.xlabel("Mini-epochs")
plt.ylabel("Dice Loss")
plt.plot(x_values, train_scores1, color="blue", label="train")
plt.plot(x_values, val_scores1, color="orange", label="val")
plt.legend()
plt.grid()

domain = len(train_scores2)
a2 = plt.subplot(2,2,4)
a2.set_ylim([0, 0.5])
plt.title("Tied Simple AENet: 7,117,071 params")
plt.xlabel("Mini-epochs")
plt.ylabel("Dice Loss")
plt.plot(x_values, train_scores2, color="blue", label="train")
plt.plot(x_values, val_scores2, color="orange", label="val")
plt.legend()
plt.grid()


plt.tight_layout()
plt.show()
plt.close()